# Import and Setup

In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
# Using np.random.uniform we generate 2000 random values within the range low and high to use as lat and long
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list which we can loop through later
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Create a list to hold the cities.
cities = []
# Loop through our list of coordinates and obtain the city name by using the citipy module
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

759

# API Call

In [5]:
# Import the requests library to be able to call the API
import requests

# Import the API key from our config.py file 
from config import weather_api_key

# Starting URL for Weather Map API Call.
# when making the call we need to adhere to the format specified by the site
# i.e. https://openweathermap.org/current
# i.e. one of the specifications is unit which can be imperial, metric or standard 
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Loop through our list of cities 
for i, city in enumerate(cities):
    # Define the city_name for the query, and replace spaces with "+" so the URL works 
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Run an API request for each of the cities.
    try:
        # Get the info from the URL and store it in json format
        city_weather = requests.get(city_url).json()
        # Country
        city_country = city_weather["sys"]["country"]
        # Latitude and longitude
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        # Maximum temperature
        city_max_temp = city_weather["main"]["temp_max"]
        # Percent humidity
        city_humidity = city_weather["main"]["humidity"]
        # Percent cloudiness
        city_clouds = city_weather["clouds"]["all"]
        # Wind speed
        city_wind = city_weather["wind"]["speed"]
        # Weather description (for example, clouds, fog, light rain, clear sky)
        city_description = city_weather['weather'][0]['description']
        # Append the city information into city_data list.
        # We append the city_data with a dictionary for that city, with the key:value pairs defined as follows 
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":city_description})

# If an error is experienced, skip the city.
    # If there is no weather data for the city, i.e. , a <Response [404]> then there is no weather to retrieve 
    # City not found. Skipping... is printed.
    # pass statement, which is a general purpose statement to handle all errors encountered and to allow the program to continue
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
# closing print statement, which will let us know the data retrieval has been completed
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. Skipping...
City not found. 

In [7]:
# Check that we returned results 
# It would make sense for this to be slightly smaller than the original city list since some won't have results found
len_city_data = len(city_data)
len_city_data

704

# Creating and Exporting the DataFrame

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Rikitea,PF,-23.1203,-134.9692,78.82,73,100,18.32,light rain
1,Port Shepstone,ZA,-30.7414,30.4550,71.74,72,100,5.12,overcast clouds
2,Arraial Do Cabo,BR,-22.9661,-42.0278,74.26,74,18,19.24,few clouds
3,Comodoro Rivadavia,AR,-45.8667,-67.5000,53.60,87,75,2.30,light intensity drizzle
4,Talara,PE,-4.5772,-81.2719,71.46,77,26,12.62,scattered clouds
5,Ribeira Grande,PT,38.5167,-28.7000,58.48,77,13,14.09,few clouds
6,Olinda,BR,-8.0089,-34.8553,78.80,78,40,4.61,scattered clouds
7,Palwal,IN,28.1500,77.3333,87.80,19,0,9.22,dust
8,Brownwood,US,31.7093,-98.9912,39.20,65,75,8.12,broken clouds
9,Victoria,HK,22.2855,114.1577,82.99,77,73,1.01,broken clouds


In [9]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
# The last line in the code block will export the DataFrame to a CSV file, with the index label (or column A) header as "City_ID." 
# If we ever need to export the CSV file to a DataFrame, that header will be present in the DataFrame
city_data_df.to_csv(output_data_file, index_label="City_ID")